In [3]:
import requests
import pandas as pd
import time
import requests
import requests.auth
import praw
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
import nltk

#import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import requests
import time, os
from bs4 import BeautifulSoup as bs


#saving
#pickle.dump(master_df, open("master_df.pickle", "wb" ))

#reading df
#submissions_df=pickle.load(open("submissions_df.pickle", "rb"))
comments_df=pickle.load(open("comments_df.pickle", "rb"))


%run text_preprocessing

['Blood', 'test', 'for', 'Down', "'s", 'syndrome', 'hailed']
['blood', 'test', 'syndrome', 'hailed']


In [14]:
#print(comments_df.columns, submissions_df.columns)


# create sample of full df
sample_df = comments_df['body'].sample(10000)

sample_uncleaned = comments_df['body'].sample(10000)

# remove urls up here so they're caught by the filter in next chunk
sample_df=sample_df.apply(remove_URL)

# find which values are most repeated. these are removed 
pd.DataFrame(sample_df).value_counts(ascending=False).reset_index().head(50)


#sample_df[sample_df.str.contains('bot')]

,body,0
0,[removed],1455
1,[deleted],147
2,Your [text post]( was removed for not having a...,99
3,Your comment was removed because it contains a...,87
4,"Hello, your post was removed because your acco...",38
5,"If this submission was flaired inaccurately, [...",30
6,Hello!\n\nDirect links to image or video posts...,29
7,* **[Bitcoin]( [Pros]( &amp; [Cons]( - Partici...,24
8,\n\nHere is the link to that Twitter thread on...,15
9,It appears you are trying to submit a post wit...,11


#### Remove rows where data is deleted, removed, or a bot response. Any responses with more than two responses will be removed. This number might change when using full dataset 

In [15]:
# save value counts
value_counts = sample_df.value_counts()
# get indices of those with value counts greater than or equal to 2
to_remove = value_counts[value_counts >=2].index
# save df as df with all data filtered out
sample_df = sample_df[~sample_df.isin(to_remove)]

#check to make sure looks good
pd.DataFrame(sample_df).value_counts(ascending=False).reset_index()


,body,0
0,\n\n\n\nhere two examples how they go after mo...,1
1,Simple advice. Buy and then forget. Only check...,1
2,Silly because I'm certain it's about to move u...,1
3,Sigh. \n\n“Nobody really knows…” and “start of...,1
4,Sigh,1
...,...,...
7888,I got the same letter. No idea which service l...,1
7889,I got the cc a few months back and with the ca...,1
7890,I got on the CRO wagon as soon as it hit Coinb...,1
7891,"I got into ALGO for the rewards, the team behi...",1


In [16]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

# PREPROCESSING

In [17]:
stemmer = SnowballStemmer("english")

In [18]:
vec = CountVectorizer(stop_words='english',
                      min_df=2,  # This is default; this is just a reminder it exists
                      max_df=1.0)



In [19]:
doc_term = vec.fit_transform(sample_df)

doc_term.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [20]:
doc_term.shape

(7893, 5302)

In [21]:
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from nltk.tokenize import WhitespaceTokenizer
import re
import string
from nltk.tokenize import RegexpTokenizer, word_tokenize
#import spacy
import gensim

#might not use these
#sp = spacy.load('en_core_web_sm')
#all_stopwords = sp.Defaults.stop_words
#all_stopwords = stopwords.words('english')
#all_stopwords.remove('not')

all_stopwords = gensim.parsing.preprocessing.STOPWORDS


In [22]:
stemmer = SnowballStemmer("english")
whitespace_tokenizer = WhitespaceTokenizer()


sample_df=sample_df.apply(replace_contractions)

#sample_df=sample_df.apply(remove_URL) # already did this
sample_df=sample_df.map(lambda x:re.sub('[%s]' % re.escape(string.punctuation), ' ', x))
sample_df=sample_df.map(lambda x: x.lower())
sample_df=sample_df.apply(word_tokenize)
sample_df=[[word for word in sentence if not word in all_stopwords] for sentence in sample_df]
sample_df=[" ".join(sentence) for sentence in sample_df]
sample_df=[[stemmer.stem(word) for word in whitespace_tokenizer.tokenize(sentence)] for sentence in sample_df]
sample_df=[remove_numbers(sentence) for sentence in sample_df]
sample_df=[" ".join(sentence) for sentence in sample_df]



 Use code below of want to make changes on a series **within** a df

stemmer = SnowballStemmer("english")
whitespace_tokenizer = WhitespaceTokenizer()


sample_df['clean_body']=sample_df['body'].apply(replace_contractions)
sample_df['clean_body']=sample_df['clean_body'].apply(remove_URL)
sample_df['clean_body']=sample_df['clean_body'].map(lambda x:re.sub('[%s]' % re.escape(string.punctuation), ' ', x))
sample_df['clean_body']=sample_df['clean_body'].map(lambda x: x.lower())
sample_df['clean_body']=sample_df['clean_body'].apply(word_tokenize)
sample_df['clean_body']=[[word for word in sentence if not word in all_stopwords] for sentence in sample_df['clean_body']]
sample_df['clean_body']=[" ".join(sentence) for sentence in sample_df['clean_body']]
sample_df['clean_body']=[[stemmer.stem(word) for word in whitespace_tokenizer.tokenize(sentence)] for sentence in sample_df['clean_body']]
sample_df['clean_body']=[" ".join(sentence) for sentence in sample_df['clean_body']]



cv = CountVectorizer(stop_words='english', lowercase=True, min_df = 2, ngram_range = (1,3))
#applying fit_transform on review
X_vect = cv.fit_transform(sample_df['clean_body']).toarray()
#converting to df and naming columns and 
X_vect_df=pd.DataFrame(X_vect,index=sample_df['clean_body'],columns=cv.get_feature_names())


In [25]:
#scroll through page to get all data (bc java...)

#driver.quit()
url = "https://coinmarketcap.com/all/views/all/"
driver = webdriver.Safari()
driver.get(url)
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.page_source


y = 1000
for timer in range(0,50):
    driver.execute_script("window.scrollTo(0, "+str(y)+")")
    y += 1000 
    time.sleep(1)
    
    


In [32]:
# soup the page 
soup = bs(driver.page_source)
#get list of all crypto names and symbols
cryp_list = []

### save symbol and name in list created 
for div in soup.find_all('tr'):
    if div.find(class_= "cmc-table__column-name--symbol") != None:
        symbol = div.find(class_= "cmc-table__column-name--symbol").text.lower()
        if div.find(class_= "cmc-table__column-name--name") != None:
            name = div.find(class_= "cmc-table__column-name--name").text.lower()
            cryp_list.append([symbol, name])


#driver.quit()
#cryp_list

In [37]:
#save all comments that mention a crypto be name or symbol to dict where key is the crypto symbol

sample_df=pd.DataFrame(sample_df)

a_dict = {}
for index,cryp in enumerate(cryp_list):
    a_dict[cryp[0]] = sample_df[sample_df[0].str.contains("\\b"+cryp_list[index][0]+"\\b|"+cryp_list[index][1]+"\\b")]
    
#get rid of empty keys
a_dict=a_dict={k:v for (k,v) in a_dict.items() if not v.empty}

In [34]:
#create x_vect for each crypto using the crypto names in a_dict

vect_dict={}
for index,key in enumerate(a_dict):
    if len(key)>=1: 
        cv = CountVectorizer(stop_words='english', ngram_range = (1,2))
        vect_dict[key] = pd.DataFrame(cv.fit_transform(a_dict[key][0]).toarray(),
                 index=a_dict[key][0], 
                 columns = cv.get_feature_names())
        
    


# uncleaned dict for sentiment analysis

In [49]:
#save all comments that mention a crypto be name or symbol to dict where key is the crypto symbol

sample_uncleaned=pd.DataFrame(sample_uncleaned)

uncleaned_dict = {}
for index,cryp in enumerate(cryp_list):
    uncleaned_dict[cryp[0]] = sample_uncleaned[sample_uncleaned['body'].str.contains("\\b"+cryp_list[index][0]+"\\b|"+cryp_list[index][1]+"\\b")]
    
#get rid of empty keys
uncleaned_dict=uncleaned_dict={k:v for (k,v) in uncleaned_dict.items() if not v.empty}

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


# Sentiment analysis

- one idea would be to see sentiment analysis for different cryptos
     - would need to extract name and symbol and do sent analysis for each

# left off here night of 12/2

In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

analyzer = SentimentIntensityAnalyzer() 

#can check out sentiment on each crypto
#see what the sentiment is on shiba inu
analyzer.polarity_scores(uncleaned_dict['shib']['body'])


# if want to check out sentiment on individual sentences 
#[analyzer.polarity_scores(x) for x in a_dict['shib'][0]]
   


{'neg': 0.122, 'neu': 0.777, 'pos': 0.101, 'compound': -0.9793}

# Topic Modeling

## LSA for all data

In [760]:
cv = CountVectorizer(stop_words='english', ngram_range = (1,2))
#applying fit_transform on review
X_vect = cv.fit_transform(sample_df[0]).toarray()
#converting to df and naming columns and 
X_vect_df=pd.DataFrame(X_vect,index=sample_df,columns=cv.get_feature_names())


lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(X_vect_df)
lsa.explained_variance_ratio_

### YOUR CODE HERE
topic_term=lsa.components_


output=display_topics(lsa,cv.get_feature_names(),10)
### OUTPUT THE CONTENTS OF THE VARIABLE
output;

topic_term_df;

pd.DataFrame(doc_topic.round(5),
             index = pd.Series(sample_df[0]),
             columns = ["component_1","component_2" ])


Topic  1
crypto, like, coin, peopl, buy, market, money, think, time, know

Topic  2
tax, pay, cost, peopl, govern, like, need, inflat, pay tax, use




,component_1,component_2
0,,
three coin select,0.26850,-0.32853
wish abl use brave start grad school whale lol,0.24724,0.14792
gt mad rule way hold control freudian slip,0.17628,0.07825
fuck zuck,0.02406,0.01293
care said,0.05281,0.02111
...,...,...
hollllyyyyyy thought,0.01599,0.00816
,0.00000,0.00000
binanc sketchi use,0.10921,0.07641


## LSA function

In [838]:
def lsa_fxn(data, dimensions):
    cv = CountVectorizer(stop_words='english', ngram_range = (1,2))
    #applying fit_transform on review
    X_vect = cv.fit_transform(data).toarray()
    #converting to df and naming columns and 
    X_vect_df=pd.DataFrame(X_vect,index=data,columns=cv.get_feature_names())
    #instantiate lsa
    lsa = TruncatedSVD(dimensions)
    #transform the X_vect_df 
    doc_topic = lsa.fit_transform(X_vect_df)
    # how much each term belongs to the different components
    topic_term=lsa.components_
    # most common word/phrase for each component
    output=display_topics(lsa,cv.get_feature_names(),10)
    # df for each term belongs to the different components
    topic_term_df = pd.DataFrame(doc_topic.round(5),
             index = pd.Series(data),
             columns = ["component_1","component_2" ])
    
    return(topic_term_df)


## LSA fxn applied to a cryp

In [841]:
lsa_fxn(a_dict['btc'][0], 2)




Topic  1
chang, hour, chang hour, btc, eth, new, bch, ada, ltc, doge

Topic  2
bitcoin, btc, podcast, money, good, market, great, book, crypto, coin




,component_1,component_2
0,,
tri buy bitcoin btc buy cost percentag countri gdp,0.05494,1.06307
btc 70k end day clear gif giphi xt39d2ujcj663gkpki,0.03894,0.18801
gt op suggest tether entir point comment use tether order tether abl control valu bitcoin earli octob billion tether creat exchang pump price reason actual whale want buy open market inflat price arrang otc trade avoid slippag clear wash trade pump price given know fact tether fraudul creat million tether strong reason suspect creat billion given major crypto trade tether liquid current price like high inflat fraudul demand like hous market sub prime mortgag fraud mean tether goe boom major valu money fantasi total control currenc definit social construct technolog currenc total control control currenc own control person useless,0.09500,2.91106
get kid tradit save account yesterday lesson memecoin ’ valu plummet chore btc,0.03794,0.23642
btc eth ergo look nicehash miner chang profit coin pay btc,0.68232,0.56220
...,...,...
like bitcoin halv,0.01552,0.82260
69k btc real,0.03636,0.18524
btc 100k,0.03611,0.15219


# Non-negative matrix factorization

In [319]:
### YOUR CODE HERE

nmf = NMF(2, init='nndsvda') 
nmf.fit_transform(X_vect_df)

### YOUR CODE HERE
topic_term=nmf.components_ 

In [320]:
topic_word = pd.DataFrame(nmf.components_.round(3),
             index = ["component_1","component_2"],
             columns = cv.get_feature_names())
topic_word


,00,000,10,10 alt,100,100x,101,10k,10x,1200,...,yeah exact,yeah need,year,yep,yes,yesterday,yield,york,youtub,zero
component_1,0.000,0.007,0.222,0.002,0.009,0.003,0.004,0.005,0.004,0.034,...,0.000,0.002,0.033,0.001,0.003,0.000,0.002,0.000,0.000,0.006
component_2,0.006,0.077,0.055,0.019,0.047,0.003,0.006,0.062,0.008,0.007,...,0.006,0.003,0.416,0.004,0.087,0.015,0.087,0.059,0.008,0.299


In [321]:
display_topics(nmf, cv.get_feature_names(),5);



Topic  1
node, network, comput, power, run

Topic  2
tax, money, like, hold, time




/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [831]:
a_dict['btc']

,0
5,tri buy bitcoin btc buy cost percentag countri...
46,btc 70k end day clear gif giphi xt39d2ujcj663g...
49,gt op suggest tether entir point comment use t...
56,get kid tradit save account yesterday lesson m...
64,btc eth ergo look nicehash miner chang profit ...
...,...
7618,like bitcoin halv
7643,69k btc real
7647,btc 100k
7725,pretti select friend talk crypto prefac conver...


In [165]:
sample_df[5]
test=pd.DataFrame(sample_df)

TypeError: center expected at least 1 argument, got 0

Mel is there


In [207]:

string_length=len(string3)+2   # will be adding 10 extra spaces

string3.center(string_length)


string_length

52

In [301]:
#searches for either symbol or text of crypto together
#(test[test[0].str.contains('|'.join(cryp_list[5]))])



pd.DataFrame(sample_df)[pd.DataFrame(sample_df)[0].str.contains(r'\b{}\b'.format('|'.join(cryp_list[0])))]
 


,0
2,print bitcoin guy understand fundament shift o...
21,current like 10 wait new ath rid worth opinion...
26,environment friend plant tree shut mine oper e...
33,canadian 107 btc
91,bunch 160 yes regret bought bitcoin went like ...
100,okay help seen help comment thing specif ask g...
106,stop read creator bitcoin
118,bitcoin blümel coin
122,ada x potenti btc safeti unpopular opinion eth...
153,gt compar said march 2020 pretti explicit say ...


In [235]:
cryp_list[0][0]+"|"+cryp_list[0][1]

'btc|bitcoin'

In [272]:
r'\b'.format('|'.join(cryp_list[0]))

'\\b'

In [254]:
mask = test[test[0].str.contains(r'\b(?:{})\b'.format('|'.join(words)))
#df1 = df1[~mask]


SyntaxError: unexpected EOF while parsing (<ipython-input-254-46eb7832be0e>, line 2)

In [260]:
(pd.DataFrame(cryp_list).iloc[0,:])

0        btc
1    bitcoin
Name: 0, dtype: object

In [271]:
cryp_list[0]

['btc', 'bitcoin']